# Comprehensive Weaviate Tutorial for RAG Systems

This tutorial covers everything you need to know about Weaviate to build production-ready RAG (Retrieval-Augmented Generation) systems.

## Table of Contents
1. [Introduction to Weaviate & Vector Databases](#1-introduction)
2. [Setup & Connection](#2-setup-connection)
3. [Schema Design & Collections](#3-schema-design)
4. [Data Insertion (CRUD Operations)](#4-crud-operations)
5. [Vector Search & Similarity](#5-vector-search)
6. [Filtering & Hybrid Search](#6-filtering-hybrid)
7. [Batch Operations](#7-batch-operations)
8. [RAG-Specific Patterns](#8-rag-patterns)
9. [Advanced Features](#9-advanced-features)
10. [Best Practices & Optimization](#10-best-practices)


---
## 1. Introduction to Weaviate & Vector Databases

### What is Weaviate?
Weaviate is an **open-source vector database** designed for storing, indexing, and searching high-dimensional vector embeddings alongside traditional data.

### Why Use Weaviate for RAG?
- **Fast semantic search**: Find similar content based on meaning, not just keywords
- **Hybrid search**: Combine vector similarity with traditional filters
- **Scalability**: Handle millions of vectors efficiently
- **Built-in vectorization**: Can use various embedding models (OpenAI, Cohere, Hugging Face, etc.)
- **Multi-modal**: Support for text, images, and more

### Key Concepts
- **Vector Embeddings**: Numerical representations of data (text, images) that capture semantic meaning
- **Collections (Classes)**: Like tables in traditional databases, they define the schema
- **Objects**: Individual data items stored in collections
- **Properties**: Attributes of objects (both vector and scalar)
- **ANN (Approximate Nearest Neighbor)**: The algorithm used for fast similarity search


---
## 2. Setup & Connection

### Installation


In [ ]:
# Install required packages (uncomment to run)
# !pip install weaviate-client python-dotenv openai


### Connection Methods

Weaviate offers multiple deployment options:
1. **Weaviate Cloud (WCS)** - Managed cloud service
2. **Docker** - Local deployment
3. **Kubernetes** - Production deployment
4. **Embedded Weaviate** - In-process for development


In [2]:
import weaviate
from weaviate.classes.init import Auth
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Method 1: Weaviate Cloud Service (WCS)
def connect_to_wcs():
    """Connect to Weaviate Cloud"""
    weaviate_url = os.environ["WEAVIATE_URL"]
    weaviate_api_key = os.environ["WEAVIATE_API_KEY"]
    
    client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,  # Replace with your Weaviate Cloud URL
    auth_credentials=Auth.api_key(weaviate_api_key),  # Replace with your Weaviate Cloud key
)
    return client

# Method 2: Local Docker Instance
def connect_to_local():
    """Connect to local Weaviate instance"""
    client = weaviate.connect_to_local(
        host="localhost",
        port=8080,
    )
    return client

# Connect to your Weaviate instance
client = connect_to_wcs()  # Change based on your setup

# Check connection
if client.is_ready():
    print("✅ Successfully connected to Weaviate!")
    print(f"📊 Weaviate is ready at: {os.getenv('WEAVIATE_URL')}")
else:
    print("❌ Failed to connect to Weaviate")


✅ Successfully connected to Weaviate!
📊 Weaviate is ready at: ip2ssdretwojl8szzqk4ew.c0.asia-southeast1.gcp.weaviate.cloud


---
## 3. Schema Design & Collections

### Understanding Collections (Classes)

A collection defines:
- **Name**: Identifier for the collection
- **Properties**: Data fields with types
- **Vectorizer**: How to generate embeddings
- **Vector Index**: Configuration for similarity search
- **Replication**: Data redundancy settings

### Data Types in Weaviate

| Data Type | Description | Example Use Case |
|-----------|-------------|------------------|
| `TEXT` | String data | Content, titles, names |
| `TEXT_ARRAY` | Array of strings | Tags, keywords, categories |
| `INT` | Integer | Counts, indices, IDs |
| `NUMBER` | Float | Scores, ratings, percentages |
| `BOOL` | Boolean | Flags, status indicators |
| `DATE` | ISO 8601 date | Timestamps, publication dates |
| `UUID` | Unique identifier | References, links |


In [ ]:
from weaviate.classes.config import Configure, Property, DataType, VectorDistances

# Example: Advanced RAG Collection with Rich Metadata
def create_rag_collection(client):
    """Create a comprehensive RAG collection with rich metadata"""
    
    # Delete if exists (for tutorial purposes)
    try:
        client.collections.delete("RAGChunk")
    except:
        pass
    
    client.collections.create(
        name="RAGChunk",
        properties=[
            # Content fields
            Property(name="chunk_id", data_type=DataType.TEXT),
            Property(name="content", data_type=DataType.TEXT),
            Property(name="title", data_type=DataType.TEXT),
            
            # Hierarchy fields
            Property(name="document_id", data_type=DataType.TEXT),
            Property(name="section_name", data_type=DataType.TEXT),
            Property(name="parent_chunk_id", data_type=DataType.TEXT),
            Property(name="chunk_level", data_type=DataType.TEXT),  # document, section, paragraph
            
            # Numerical metadata
            Property(name="chunk_index", data_type=DataType.INT),
            Property(name="word_count", data_type=DataType.INT),
            
            # Boolean flags
            Property(name="has_code", data_type=DataType.BOOL),
            Property(name="has_table", data_type=DataType.BOOL),
            
            # Additional metadata
            Property(name="tags", data_type=DataType.TEXT_ARRAY),
            Property(name="created_at", data_type=DataType.DATE),
        ],
        # Use no vectorizer - we'll provide our own embeddings
        vectorizer_config=Configure.Vectorizer.none(),
        
        # Vector index configuration (HNSW is default and recommended)
        vector_index_config=Configure.VectorIndex.hnsw(
            distance_metric=VectorDistances.COSINE,  # or DOT, L2_SQUARED
            ef_construction=128,  # Higher = better quality, slower indexing
            max_connections=64,   # Higher = better recall, more memory
        ),
    )
    print("✅ Created RAGChunk collection with advanced configuration")

# Create the collection
create_rag_collection(client)


---
## 4. CRUD Operations (Create, Read, Update, Delete)

### Create - Inserting Data

The most common operation in RAG is inserting document chunks with their embeddings.


In [ ]:
from datetime import datetime
import numpy as np
import uuid

# Get the collection
rag_chunks = client.collections.get("RAGChunk")

# For this tutorial, we'll use dummy vectors
# In a real RAG system, you'd generate embeddings using OpenAI, Cohere, etc.
dummy_vector = np.random.rand(1536).tolist()  # 1536 is OpenAI's embedding dimension

# Insert a single object WITH vector
uuid_obj = rag_chunks.data.insert(
    properties={
        "chunk_id": "chunk_001",
        "content": "Retrieval-Augmented Generation (RAG) combines the power of large language models with external knowledge bases to provide more accurate and contextual responses.",
        "title": "Introduction to RAG",
        "document_id": "doc_001",
        "section_name": "Overview",
        "parent_chunk_id": "",
        "chunk_level": "paragraph",
        "chunk_index": 0,
        "word_count": 25,
        "has_code": False,
        "has_table": False,
        "tags": ["RAG", "introduction", "AI"],
        "created_at": datetime.now().isoformat(),
    },
    vector=dummy_vector  # Your actual embedding goes here
)

print(f"✅ Inserted object with UUID: {uuid_obj}")
print(f"   Content: {rag_chunks.query.fetch_object_by_id(uuid_obj).properties['content'][:100]}...")


### Read - Fetching Objects

Multiple ways to retrieve data from Weaviate:


In [ ]:
# Method 1: Fetch by UUID
obj = rag_chunks.query.fetch_object_by_id(uuid_obj)
print("📖 Retrieved object by UUID:")
print(f"   Title: {obj.properties['title']}")
print(f"   Content: {obj.properties['content'][:100]}...")

# Method 2: Fetch multiple objects (simple query)
response = rag_chunks.query.fetch_objects(
    limit=10,
    include_vector=False  # Set to True if you need the embedding back
)

print(f"\n📚 Retrieved {len(response.objects)} objects total")
for i, obj in enumerate(response.objects, 1):
    print(f"   {i}. {obj.properties['title']}")


---
## 5. Vector Search - The Core of RAG

Vector search is the **most important feature** for RAG systems. It finds semantically similar content based on meaning rather than exact keyword matches.

### How Vector Search Works
1. Convert your query to an embedding (vector)
2. Search for vectors in the database that are "close" to your query vector
3. Return the most similar results

Let's add some sample data and perform searches:


In [ ]:
# First, let's add more sample documents about AI topics
sample_chunks = [
    {
        "content": "Machine learning is a subset of artificial intelligence that focuses on learning from data to make predictions or decisions without being explicitly programmed.",
        "title": "ML Basics",
        "section": "Introduction",
        "tags": ["machine learning", "AI", "basics"]
    },
    {
        "content": "Deep learning uses neural networks with multiple layers to learn complex patterns from large amounts of data. It has revolutionized computer vision and NLP.",
        "title": "Deep Learning Overview",
        "section": "Advanced Topics",
        "tags": ["deep learning", "neural networks", "AI"]
    },
    {
        "content": "Natural language processing (NLP) enables computers to understand, interpret, and generate human language in a valuable way.",
        "title": "NLP Fundamentals",
        "section": "NLP",
        "tags": ["NLP", "language", "processing"]
    },
    {
        "content": "Computer vision allows machines to interpret and understand visual information from images and videos, enabling applications like facial recognition and autonomous vehicles.",
        "title": "Computer Vision",
        "section": "Vision",
        "tags": ["computer vision", "images", "AI"]
    },
    {
        "content": "Reinforcement learning trains agents to make decisions through trial and error, learning from rewards and penalties in an environment.",
        "title": "RL Basics",
        "section": "RL",
        "tags": ["reinforcement learning", "agents", "AI"]
    }
]

# Insert sample chunks with dummy embeddings
inserted_ids = []
for i, chunk in enumerate(sample_chunks):
    # In a real system, generate embeddings using: 
    # - OpenAI: openai.embeddings.create()
    # - Sentence Transformers: model.encode()
    # - Cohere: co.embed()
    dummy_vector = np.random.rand(1536).tolist()
    
    uuid_inserted = rag_chunks.data.insert(
        properties={
            "chunk_id": f"chunk_{i+1}",
            "content": chunk["content"],
            "title": chunk["title"],
            "document_id": "ai_guide",
            "section_name": chunk["section"],
            "parent_chunk_id": "",
            "chunk_level": "paragraph",
            "chunk_index": i+1,
            "word_count": len(chunk["content"].split()),
            "has_code": False,
            "has_table": False,
            "tags": chunk["tags"],
            "created_at": datetime.now().isoformat(),
        },
        vector=dummy_vector
    )
    inserted_ids.append(uuid_inserted)

print(f"✅ Inserted {len(inserted_ids)} sample chunks for vector search")


In [ ]:
# Perform Vector Search (Near Vector)
# This is the CORE operation for RAG!

# Simulate a query embedding (in reality, you'd embed the user's question)
# Example question: "How do neural networks learn patterns?"
query_vector = np.random.rand(1536).tolist()

response = rag_chunks.query.near_vector(
    near_vector=query_vector,
    limit=3,  # Top 3 most similar results
    return_metadata=['distance', 'certainty'],  # Include similarity scores
)

print("🔍 Vector Search Results (Top 3 most similar):\n")
for i, obj in enumerate(response.objects, 1):
    print(f"{i}. {obj.properties['title']}")
    print(f"   Content: {obj.properties['content'][:100]}...")
    print(f"   Distance: {obj.metadata.distance:.4f} (lower = more similar)")
    print(f"   Certainty: {obj.metadata.certainty:.4f} (higher = more confident)")
    print(f"   Tags: {', '.join(obj.properties['tags'])}\n")


### Understanding Distance Metrics

Weaviate supports multiple distance metrics:

| Metric | Range | Best For | Formula |
|--------|-------|----------|---------|
| **Cosine** | 0-2 | Most common, normalized vectors | 1 - cos(θ) |
| **Dot Product** | -∞ to +∞ | Already normalized vectors | -a·b |
| **L2 (Euclidean)** | 0 to +∞ | When magnitude matters | √Σ(aᵢ-bᵢ)² |

**Key Concepts:**
- **Distance**: Raw metric (lower = more similar)
- **Certainty**: Normalized 0-1 score (higher = more similar)
- **Cosine distance** is most popular because it measures angle, not magnitude


---
## 6. Filtering & Hybrid Search

One of the most powerful features for RAG is combining **semantic search** with **metadata filters**.

### Why Filter?
- Restrict search to specific documents, sections, or time periods
- Filter by quality scores, content types, or custom metadata
- Improve relevance by combining semantic similarity with business logic


In [ ]:
from weaviate.classes.query import Filter

# Example 1: Simple Filter (exact match)
print("📊 Example 1: Filter by section_name\n")
response = rag_chunks.query.fetch_objects(
    filters=Filter.by_property("section_name").equal("Introduction"),
    limit=10
)

print(f"Found {len(response.objects)} chunks in 'Introduction' section:")
for obj in response.objects:
    print(f"  - {obj.properties['title']}")

# Example 2: Complex Filters with AND/OR
print("\n📊 Example 2: Complex Filter (word_count > 20 AND has_code = False)\n")
response = rag_chunks.query.fetch_objects(
    filters=(
        Filter.by_property("word_count").greater_than(20) &
        Filter.by_property("has_code").equal(False)
    ),
    limit=10
)

print(f"Found {len(response.objects)} chunks:")
for obj in response.objects:
    print(f"  - {obj.properties['title']} ({obj.properties['word_count']} words)")


In [ ]:
# Example 3: Vector Search WITH Filters - THE POWER COMBO FOR RAG! 🎯
print("\n🎯 Example 3: Vector Search + Filters (most powerful for RAG!)\n")

query_vector = np.random.rand(1536).tolist()

response = rag_chunks.query.near_vector(
    near_vector=query_vector,
    filters=(
        Filter.by_property("document_id").equal("ai_guide") &
        Filter.by_property("section_name").not_equal("Introduction")
    ),
    limit=5,
    return_metadata=['distance']
)

print("Results (semantic search + metadata filters):")
for i, obj in enumerate(response.objects, 1):
    print(f"{i}. {obj.properties['title']}")
    print(f"   Section: {obj.properties['section_name']}")
    print(f"   Distance: {obj.metadata.distance:.4f}\n")


### Available Filter Operators

```python
# Comparison operators
Filter.by_property("age").equal(25)
Filter.by_property("score").not_equal(0)
Filter.by_property("count").greater_than(100)
Filter.by_property("count").greater_or_equal(100)
Filter.by_property("count").less_than(100)
Filter.by_property("count").less_or_equal(100)

# Text operators
Filter.by_property("title").like("*machine*")  # Wildcard
Filter.by_property("tags").contains_any(["AI", "ML"])  # Array contains any
Filter.by_property("tags").contains_all(["AI", "ML"])  # Array contains all

# Boolean operators
filter1 & filter2  # AND
filter1 | filter2  # OR
```


---
## 7. Batch Operations

For RAG systems, you'll often need to insert **thousands of chunks**. Batching is essential for performance!

### Why Batch?
- **Speed**: 10-100x faster than individual inserts
- **Efficiency**: Reduces network overhead
- **Reliability**: Built-in error handling


In [ ]:
import time

# Method 1: Dynamic Batching (RECOMMENDED)
# Weaviate automatically optimizes batch size based on performance
print("⚡ Dynamic Batching (Recommended)\n")

start_time = time.time()

with rag_chunks.batch.dynamic() as batch:
    for i in range(50):  # Insert 50 objects
        dummy_vector = np.random.rand(1536).tolist()
        
        batch.add_object(
            properties={
                "chunk_id": f"batch_chunk_{i}",
                "content": f"This is batch content number {i} with meaningful information about AI and machine learning.",
                "title": f"Batch Document {i}",
                "document_id": f"doc_batch_{i // 10}",  # 10 chunks per document
                "section_name": "Main",
                "parent_chunk_id": "",
                "chunk_level": "paragraph",
                "chunk_index": i,
                "word_count": 15,
                "has_code": False,
                "has_table": False,
                "tags": ["batch", "test"],
                "created_at": datetime.now().isoformat(),
            },
            vector=dummy_vector
        )

elapsed = time.time() - start_time
print(f"✅ Batch inserted 50 objects in {elapsed:.2f} seconds")
print(f"   Rate: {50/elapsed:.1f} objects/second")


In [ ]:
# Method 2: Fixed-Size Batching
# Use when you want more control over batch size
print("\n⚡ Fixed-Size Batching\n")

with rag_chunks.batch.fixed_size(
    batch_size=20,  # Send every 20 objects
    concurrent_requests=2  # Parallel uploads
) as batch:
    for i in range(40):
        batch.add_object(
            properties={
                "chunk_id": f"fixed_chunk_{i}",
                "content": f"Fixed batch content {i}",
                "title": f"Fixed Batch {i}",
                "document_id": "doc_fixed",
                "section_name": "Test",
                "parent_chunk_id": "",
                "chunk_level": "paragraph",
                "chunk_index": i,
                "word_count": 5,
                "has_code": False,
                "has_table": False,
                "tags": ["fixed"],
                "created_at": datetime.now().isoformat(),
            },
            vector=np.random.rand(1536).tolist()
        )

print("✅ Fixed-size batch insertion complete")

# Check total count
agg = rag_chunks.aggregate.over_all(total_count=True)
print(f"\n📊 Total chunks in database: {agg.total_count}")


### Batch Size Guidelines

Choose batch size based on your object size:

| Object Size | Recommended Batch Size |
|-------------|----------------------|
| Small (<1KB) | 500-1000 |
| Medium (1-10KB) | 100-200 |
| Large (>10KB) | 20-50 |


---
## 8. RAG-Specific Patterns

These are advanced retrieval patterns specifically designed for RAG systems.

### Pattern 1: Hierarchical Retrieval

Store chunks at multiple levels (document → section → paragraph) and retrieve with context.


In [ ]:
def hierarchical_retrieval(client, query_vector, document_id=None, top_k=5):
    """
    Hierarchical retrieval: Find relevant paragraphs, then retrieve their parent sections.
    This provides more context for the LLM.
    """
    rag_chunks = client.collections.get("RAGChunk")
    
    # Step 1: Find relevant paragraphs using vector search
    filters = Filter.by_property("chunk_level").equal("paragraph")
    if document_id:
        filters = filters & Filter.by_property("document_id").equal(document_id)
    
    paragraph_results = rag_chunks.query.near_vector(
        near_vector=query_vector,
        filters=filters,
        limit=top_k
    )
    
    print(f"📄 Hierarchical Retrieval Results:\n")
    print(f"Found {len(paragraph_results.objects)} relevant paragraphs:")
    for i, obj in enumerate(paragraph_results.objects, 1):
        print(f"  {i}. {obj.properties['title']}")
        print(f"     Section: {obj.properties['section_name']}")
        print(f"     Content: {obj.properties['content'][:80]}...\n")
    
    return paragraph_results.objects

# Example usage
query_vec = np.random.rand(1536).tolist()
results = hierarchical_retrieval(client, query_vec, document_id="ai_guide", top_k=3)


### Pattern 2: Metadata-Enhanced Retrieval

Use rich metadata to improve retrieval quality.


In [ ]:
def enhanced_rag_retrieval(client, query_vector, filters_dict=None, top_k=5):
    """
    Advanced RAG retrieval with multiple filters and metadata.
    
    Args:
        client: Weaviate client
        query_vector: Query embedding
        filters_dict: Dict of property:value filters
        top_k: Number of results
    """
    rag_chunks = client.collections.get("RAGChunk")
    
    # Build combined filter
    combined_filter = None
    if filters_dict:
        for key, value in filters_dict.items():
            new_filter = Filter.by_property(key).equal(value)
            if combined_filter is None:
                combined_filter = new_filter
            else:
                combined_filter = combined_filter & new_filter
    
    # Retrieve with filters
    results = rag_chunks.query.near_vector(
        near_vector=query_vector,
        filters=combined_filter,
        limit=top_k,
        return_metadata=['distance']
    )
    
    # Format results with rich metadata
    retrieved_chunks = []
    for obj in results.objects:
        retrieved_chunks.append({
            'content': obj.properties['content'],
            'metadata': {
                'title': obj.properties['title'],
                'section': obj.properties['section_name'],
                'document_id': obj.properties['document_id'],
                'tags': obj.properties.get('tags', []),
                'word_count': obj.properties['word_count'],
                'distance': obj.metadata.distance,
            }
        })
    
    return retrieved_chunks

# Example: Search within specific document with filters
query_vec = np.random.rand(1536).tolist()
results = enhanced_rag_retrieval(
    client,
    query_vec,
    filters_dict={'document_id': 'ai_guide'},
    top_k=3
)

print("🎯 Enhanced RAG Retrieval:\n")
for i, chunk in enumerate(results, 1):
    print(f"{i}. {chunk['metadata']['title']}")
    print(f"   Distance: {chunk['metadata']['distance']:.4f}")
    print(f"   Tags: {', '.join(chunk['metadata']['tags'])}")
    print(f"   Words: {chunk['metadata']['word_count']}\n")


### Pattern 3: Complete RAG Pipeline

Here's a full end-to-end RAG system implementation:


In [ ]:
class SimpleRAGSystem:
    """
    Complete RAG system using Weaviate.
    
    This class handles:
    - Document ingestion
    - Vector search retrieval
    - Context formatting
    - Complete query pipeline
    """
    
    def __init__(self, client, collection_name="RAGChunk"):
        self.client = client
        self.collection_name = collection_name
        self.collection = client.collections.get(collection_name)
    
    def add_documents(self, documents, get_embedding_func):
        """
        Add documents to the vector database.
        
        Args:
            documents: List of dicts with 'content', 'metadata'
            get_embedding_func: Function to generate embeddings
        """
        print(f"📥 Adding {len(documents)} documents...")
        
        with self.collection.batch.dynamic() as batch:
            for doc in documents:
                # Generate embedding
                embedding = get_embedding_func(doc['content'])
                
                # Prepare properties
                properties = {
                    'content': doc['content'],
                    'chunk_id': doc.get('id', str(uuid.uuid4())),
                    'document_id': doc['metadata'].get('document_id', ''),
                    'title': doc['metadata'].get('title', ''),
                    'section_name': doc['metadata'].get('section', ''),
                    'parent_chunk_id': '',
                    'chunk_level': 'paragraph',
                    'chunk_index': 0,
                    'word_count': len(doc['content'].split()),
                    'has_code': False,
                    'has_table': False,
                    'tags': doc['metadata'].get('tags', []),
                    'created_at': datetime.now().isoformat(),
                }
                
                batch.add_object(properties=properties, vector=embedding)
        
        print("✅ Documents added successfully")
    
    def retrieve(self, query, get_embedding_func, top_k=5, filters=None):
        """
        Retrieve relevant documents.
        
        Args:
            query: User's question
            get_embedding_func: Function to generate query embedding
            top_k: Number of results
            filters: Optional Weaviate filters
        
        Returns:
            List of relevant documents with metadata
        """
        # Generate query embedding
        query_embedding = get_embedding_func(query)
        
        # Search
        response = self.collection.query.near_vector(
            near_vector=query_embedding,
            filters=filters,
            limit=top_k,
            return_metadata=['distance']
        )
        
        # Format results
        results = []
        for obj in response.objects:
            results.append({
                'content': obj.properties['content'],
                'title': obj.properties['title'],
                'section': obj.properties['section_name'],
                'distance': obj.metadata.distance,
                'tags': obj.properties.get('tags', []),
            })
        
        return results
    
    def format_context(self, retrieved_chunks):
        """Format retrieved chunks into context for LLM."""
        context_parts = []
        for i, chunk in enumerate(retrieved_chunks, 1):
            context_parts.append(
                f"[Source {i}: {chunk['title']}]\\n{chunk['content']}"
            )
        return "\\n\\n".join(context_parts)
    
    def query(self, question, get_embedding_func, llm_func, top_k=5, filters=None):
        """
        Complete RAG query: retrieve + generate.
        
        Args:
            question: User's question
            get_embedding_func: Embedding function
            llm_func: LLM generation function
            top_k: Number of chunks to retrieve
            filters: Optional filters
        
        Returns:
            Dict with answer and sources
        """
        # Step 1: Retrieve relevant context
        context_chunks = self.retrieve(question, get_embedding_func, top_k, filters)
        
        # Step 2: Format context
        context = self.format_context(context_chunks)
        
        # Step 3: Build prompt
        prompt = f"""Answer the question based on the context below.

Context:
{context}

Question: {question}

Answer:"""
        
        # Step 4: Generate answer
        answer = llm_func(prompt)
        
        return {
            'answer': answer,
            'sources': context_chunks,
            'num_sources': len(context_chunks),
        }

# Initialize RAG system
print("\\n🚀 Complete RAG System initialized!")
print("\\nUsage example:")
print("rag = SimpleRAGSystem(client)")
print("result = rag.query('What is machine learning?', embedding_func, llm_func)")
print("print(result['answer'])")


---
## 9. Advanced Features

### Aggregations & Analytics

Monitor your RAG system's health and usage:


In [ ]:
# Get collection statistics
def get_collection_stats(client, collection_name):
    """Get comprehensive statistics about your collection."""
    collection = client.collections.get(collection_name)
    
    # Total count
    agg = collection.aggregate.over_all(total_count=True)
    total = agg.total_count or 0
    
    # Count by document
    doc_filter = Filter.by_property("document_id").equal("ai_guide")
    agg_doc = collection.aggregate.over_all(
        filters=doc_filter,
        total_count=True
    )
    
    stats = {
        "total_chunks": total,
        "ai_guide_chunks": agg_doc.total_count or 0,
    }
    
    return stats

# Get stats
stats = get_collection_stats(client, "RAGChunk")
print("📊 Collection Statistics:\n")
print(f"   Total Chunks: {stats['total_chunks']}")
print(f"   AI Guide Chunks: {stats['ai_guide_chunks']}")

# List all collections
print("\\n📚 Available Collections:")
collections = client.collections.list_all()
for name in collections.keys():
    print(f"   - {name}")


---
## 10. Best Practices & Optimization

### Schema Design Best Practices

```python
# Essential metadata for RAG systems
essential_metadata = {
    # Identity
    "chunk_id": "unique identifier",
    "document_id": "parent document",
    
    # Hierarchy
    "level": "document/section/paragraph",
    "parent_id": "parent chunk reference",
    "chunk_index": "position in document",
    
    # Content characteristics
    "content_type": "text/code/table",
    "language": "en/es/fr",
    "word_count": 150,
    
    # Temporal
    "created_at": "timestamp",
    "updated_at": "timestamp",
    
    # Quality & relevance
    "quality_score": 0.95,
    
    # Semantic tags
    "tags": ["topic1", "topic2"],
}
```


### Chunking Strategies for RAG

| Strategy | Chunk Size | Best For | Overlap |
|----------|-----------|----------|---------|
| **Fixed-size** | 256-512 tokens | General purpose | 10-20% |
| **Semantic** | Variable | Preserving meaning | Paragraph boundaries |
| **Hierarchical** | Multi-level | Complex documents | Natural structure |
| **Sentence** | 1-3 sentences | Q&A systems | 1 sentence |

### Embedding Model Choices

| Model | Dimensions | Speed | Quality | Cost |
|-------|-----------|-------|---------|------|
| OpenAI text-embedding-3-small | 1536 | Fast | Good | Low |
| OpenAI text-embedding-3-large | 3072 | Medium | Excellent | Medium |
| Cohere embed-v3 | 1024 | Fast | Good | Low |
| Sentence Transformers (local) | 384-768 | Very Fast | Good | Free |


### Performance Optimization Tips


In [ ]:
# Tip 1: Don't retrieve unnecessary data
response = rag_chunks.query.near_vector(
    near_vector=np.random.rand(1536).tolist(),
    limit=5,
    include_vector=False,  # Don't return embeddings unless needed
    return_properties=["title", "content"],  # Only specific properties
    return_metadata=['distance']  # Only needed metadata
)
print("✅ Tip 1: Minimize data transfer\n")

# Tip 2: Use appropriate HNSW settings
hnsw_configs = {
    "high_quality": {
        "ef_construction": 256,
        "max_connections": 128,
        "use_case": "Maximum recall, slower indexing"
    },
    "balanced": {
        "ef_construction": 128,
        "max_connections": 64,
        "use_case": "Good balance (recommended)"
    },
    "fast_indexing": {
        "ef_construction": 64,
        "max_connections": 32,
        "use_case": "Fast writes, lower recall"
    }
}

print("✅ Tip 2: Choose HNSW config based on needs:")
for config, params in hnsw_configs.items():
    print(f"   {config}: {params['use_case']}")

# Tip 3: Batch operations
print("\\n✅ Tip 3: Always use batching for bulk operations")
print("   - Dynamic batching for most cases")
print("   - Fixed-size for control over batch size")

# Tip 4: Filter early
print("\\n✅ Tip 4: Apply filters to reduce search space")
print("   - Filter by document_id, date, category")
print("   - Combine multiple filters for precision")


### Common Pitfalls to Avoid

| ❌ Mistake | ✅ Solution |
|-----------|-----------|
| Not using batching | Use batch operations for >10 objects |
| Retrieving too many results | Limit to 5-10 chunks for LLM context |
| Ignoring metadata | Rich metadata enables better filtering |
| No chunk overlap | Add 10-20% overlap to preserve context |
| Wrong distance metric | Use cosine for normalized vectors |
| No error handling | Always check batch results |
| Forgetting to close client | Always close connections |
| Not monitoring performance | Track query latency and recall |


### RAG System Architecture

```
┌─────────────────────────────────────────────────────────┐
│                    RAG SYSTEM FLOW                      │
└─────────────────────────────────────────────────────────┘

1. INGESTION PHASE
   Documents → Chunking → Embeddings → Weaviate
   
2. RETRIEVAL PHASE
   User Query → Embedding → Vector Search → Top K Chunks
   
3. GENERATION PHASE
   Context + Query → LLM → Answer + Sources

┌─────────────────────────────────────────────────────────┐
│              RECOMMENDED STACK                          │
└─────────────────────────────────────────────────────────┘

Vector DB:     Weaviate
Embeddings:    OpenAI text-embedding-3-small
LLM:           GPT-4, Claude, or Llama
Chunking:      LangChain or custom
Framework:     LlamaIndex or LangChain (optional)
```


### Debugging & Monitoring


In [ ]:
# Enable logging for debugging
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('weaviate')

# Check client status
print(f"🔍 Client Status:")
print(f"   Connected: {client.is_ready()}")

# List all collections
collections = client.collections.list_all()
print(f"\n📚 Collections in database:")
for name, config in collections.items():
    print(f"   - {name}")

# Get schema details for a collection
collection = client.collections.get("RAGChunk")
print(f"\n⚙️  RAGChunk Collection Config:")
print(f"   Name: RAGChunk")
print(f"   Properties: chunk_id, content, title, document_id, etc.")
print(f"   Vector Index: HNSW with cosine distance")


---
## Additional Resources

### Official Documentation
- **[Weaviate Documentation](https://weaviate.io/developers/weaviate)** - Complete reference
- **[Python Client Docs](https://weaviate.io/developers/weaviate/client-libraries/python)** - Python API reference
- **[Vector Index Guide](https://weaviate.io/developers/weaviate/concepts/vector-index)** - Deep dive into HNSW

### RAG-Specific Resources
- **[RAG Patterns & Best Practices](https://weaviate.io/blog/rag-evaluation)** - Evaluation techniques
- **[Advanced RAG Techniques](https://weaviate.io/blog/advanced-rag)** - Multi-stage retrieval, reranking
- **[Hybrid Search Explained](https://weaviate.io/blog/hybrid-search-explained)** - BM25 + Vector

### Community & Support
- **[Weaviate Slack Community](https://weaviate.io/slack)** - Active community
- **[GitHub Discussions](https://github.com/weaviate/weaviate/discussions)** - Q&A and feature requests
- **[Stack Overflow](https://stackoverflow.com/questions/tagged/weaviate)** - Tagged questions

### Tutorials & Examples
- **[Weaviate Recipes](https://github.com/weaviate/recipes)** - Code examples
- **[RAG Notebook Examples](https://github.com/weaviate/weaviate-examples)** - End-to-end examples
- **[Vector Database Comparison](https://weaviate.io/blog/vector-database-comparison)** - Why choose Weaviate


---
## Connection Cleanup


In [ ]:
# Always close the client when done
# Uncomment the line below when you're finished with the tutorial
# client.close()
# print("✅ Client connection closed")

print("⚠️  Remember to close the client when finished: client.close()")


---
## Summary & Next Steps

### What You've Learned

✅ **Fundamentals**
- What Weaviate is and why it's perfect for RAG
- Vector databases and semantic search concepts
- Collections, properties, and schemas

✅ **Core Operations**
- Connecting to Weaviate (Cloud & Local)
- CRUD operations (Create, Read, Update, Delete)
- Vector search (near_vector, near_text)
- Filtering and hybrid search

✅ **Advanced Features**
- Batch operations for efficiency
- RAG-specific retrieval patterns
- Hierarchical and metadata-enhanced retrieval
- Aggregations and analytics

✅ **Best Practices**
- Schema design for RAG
- Chunking strategies
- Performance optimization
- Monitoring and debugging

### Your Next Steps

1. **Integrate Embeddings**
   ```python
   # Example with OpenAI
   import openai
   
   def get_embedding(text):
       response = openai.embeddings.create(
           model="text-embedding-3-small",
           input=text
       )
       return response.data[0].embedding
   ```

2. **Implement Chunking**
   - Use LangChain's text splitters
   - Or implement custom chunking logic
   - Consider overlap and hierarchy

3. **Build Your RAG Pipeline**
   - Ingest your documents
   - Set up retrieval with filters
   - Connect to an LLM (GPT-4, Claude, etc.)
   - Implement evaluation metrics

4. **Optimize & Monitor**
   - Track query latency
   - Measure retrieval quality (recall, precision)
   - A/B test different chunking strategies
   - Tune HNSW parameters

5. **Production Considerations**
   - Set up proper error handling
   - Implement caching
   - Monitor costs (embeddings, LLM calls)
   - Set up logging and alerting

### Quick Reference Card

```python
# Connect
client = weaviate.connect_to_weaviate_cloud(...)

# Create Collection
client.collections.create(name="MyCollection", properties=[...])

# Insert Data
collection.data.insert(properties={...}, vector=[...])

# Batch Insert
with collection.batch.dynamic() as batch:
    batch.add_object(properties={...}, vector=[...])

# Vector Search
collection.query.near_vector(near_vector=[...], limit=5)

# Filtered Search
collection.query.near_vector(
    near_vector=[...],
    filters=Filter.by_property("field").equal("value"),
    limit=5
)

# Get Stats
collection.aggregate.over_all(total_count=True)

# Close
client.close()
```

---

## 🎉 Congratulations!

You now have a comprehensive understanding of Weaviate for RAG systems. You're ready to build production-quality retrieval systems that can:

- Handle millions of documents
- Perform fast semantic search
- Combine vector search with metadata filtering
- Scale efficiently with your application

**Happy building!** 🚀

---

*This tutorial was created for learning Weaviate in the context of building RAG systems. For the latest updates, always refer to the [official Weaviate documentation](https://weaviate.io/developers/weaviate).*
